In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

In [ ]:
with open('Ações.txt') as file:
    ações = file.readlines()

ações = [ação.strip() for ação in ações]
print(ações)

In [ ]:
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

In [ ]:
def calcular_nota_ponderada(roe, cagr):
    """Calcula a nota ponderada para ROE e CAGR.

    Args:
        roe: Lista com os valores de ROE dos últimos 5 anos.
        cagr: Lista com os valores de CAGR dos últimos 5 anos.

    Returns:
        A nota ponderada para ROE e CAGR.
    """

    # Define os pesos dos indicadores
    pesos_roe = 3
    pesos_cagr = 2

    # Calcula a média ponderada dos indicadores com histórico de 5 anos
    def calcular_media_ponderada(valores):
        """Calcula a média ponderada de uma lista de valores, com maior peso para os anos mais recentes.

        Args:
            valores: Lista de valores.

        Returns:
            A média ponderada dos valores.
        """
        soma_ponderada = 0
        soma_pesos = 0
        for i, valor in enumerate(valores):
            peso = i + 1
            soma_ponderada += valor * peso
            soma_pesos += peso
        return soma_ponderada / soma_pesos

    # Calcula a pontuação de cada indicador
    pontuacao_roe = calcular_media_ponderada(roe) * pesos_roe
    pontuacao_cagr = calcular_media_ponderada(cagr) * pesos_cagr

    # Soma as pontuações
    pontuacao_total = pontuacao_roe + pontuacao_cagr

    # Retorna a nota ponderada
    return pontuacao_total


def calcular_nota_simples(margem_bruta, div_liq, lucro_liq, pl, p_vp, tag_along, free_float, fundacao, estatal, dy, payout, variacao, valor_de_mercado, ibov):
    """Calcula a nota simples para os demais indicadores.

    Args:
        margem_bruta: Valor da margem bruta.
        div_liq: Valor da dívida líquida.
        lucro_liq: Valor do lucro líquido.
        pl: Valor do P/L.
        p_vp: Valor do P/VP.
        tag_along: Valor do tag along.
        free_float: Valor do free float.
        fundacao: Ano de fundação da empresa.
        estatal:  "GOVERNO" se a empresa for estatal, "não" caso contrário.
        dy: Valor do DY.
        payout: Valor do payout.
        variacao: Lista com os valores de variação do preço da ação nos últimos 3 anos.
        valor_de_mercado: Valor de mercado da empresa.
        ibov: Participação da empresa no Ibovespa.

    Returns:
        A nota simples para os demais indicadores.
    """

    # Define os pesos dos indicadores
    pesos = {
        'margem_bruta': 2,
        'div_liq': 2,
        'lucro_liq': 2,
        'pl': 3,
        'p_vp': 1,
        'tag_along': 3,
        'free_float': 3,
        'fundacao': 3,
        'estatal': 3,
        'dy': 1,
        'payout': 1,
        'variacao': 1,
        'valor_de_mercado': 1,
        'ibov': 1
    }

    # Calcula a pontuação de cada indicador
    pontuacao_margem_bruta = margem_bruta * pesos['margem_bruta']
    pontuacao_div_liq = div_liq * pesos['div_liq']
    pontuacao_lucro_liq = lucro_liq * pesos['lucro_liq']
    pontuacao_pl = pl * pesos['pl']
    pontuacao_p_vp = p_vp * pesos['p_vp']
    pontuacao_tag_along = tag_along * pesos['tag_along']
    pontuacao_free_float = free_float * pesos['free_float']
    pontuacao_fundacao = fundacao * pesos['fundacao']
    pontuacao_estatal = 1 if estatal != "GOVERNO" else 0 * pesos['estatal']
    pontuacao_dy = dy * pesos['dy']
    pontuacao_payout = payout * pesos['payout']
    pontuacao_variacao = 1 if all(-15 < x <= 15 for x in variacao) else 0 * pesos['variacao']
    pontuacao_valor_de_mercado = 1 if valor_de_mercado > 100000000000 else 0 * pesos['valor_de_mercado']
    pontuacao_ibov = 1 if ibov > 1 else 0 * pesos['ibov']

    # Soma as pontuações
    pontuacao_total = (
        pontuacao_margem_bruta
        + pontuacao_div_liq
        + pontuacao_lucro_liq
        + pontuacao_pl
        + pontuacao_p_vp
        + pontuacao_tag_along
        + pontuacao_free_float
        + pontuacao_fundacao
        + pontuacao_estatal
        + pontuacao_dy
        + pontuacao_payout
        + pontuacao_variacao
        + pontuacao_valor_de_mercado
        + pontuacao_ibov
    )

    # Retorna a nota simples
    return pontuacao_total



In [ ]:
def scroll_to_element(driver, scroll_value, max_scroll_value, scroll_step, xpath, extra_scroll=1000):
    
    while scroll_value <= max_scroll_value:
        try:
            # Rolar a página
            driver.execute_script(f'window.scroll(0,{scroll_value});')
            # Encontrar o elemento
            element = driver.find_element(By.XPATH, xpath)
            # Encontrou o elemento, adiciona o scroll extra
            driver.execute_script(f'window.scroll(0,{scroll_value + extra_scroll});')
            # Encontrou o elemento, retorna True
            return True

        except:
            # Aumenta o valor de scroll
            scroll_value += scroll_step

    # Não encontrou o elemento, retorna False
    return False

In [ ]:

def aplicar_criterio(condicao, contador):
    
    if condicao:
        contador += 1
    else:
        contador -= 1
    return contador

In [ ]:
def aplicar_2_criterio(condicao, condicao_2, contador):
    
    if condicao:
        contador += 1
    elif condicao_2:
        contador += 0
    else:
        contador -= 1
    return contador

In [ ]:
driver = webdriver.Chrome(options=options)
#driver.maximize_window()

# Cria um DataFrame vazio para armazenar os resultados
df_resultados = pd.DataFrame(columns=['Ação', 'Nota'])

for ação in ações:
    contador = 0

    ##### fundamentus #####
    driver.get('https://www.fundamentus.com.br/detalhes.php?papel={}'.format(ação))

    div_liq = int(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[4]/tbody/tr[3]/td[4]/span').text.replace(".", "")) 
    lucro_liq = int(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[5]/tbody/tr[5]/td[2]/span').text.replace(".", "")) 
    p_vp = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[3]/td[4]/span').text.replace(",", "."))
    ebit = int(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[5]/tbody/tr[4]/td[2]/span').text.replace(".", ""))
    pl = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[2]/td[4]/span').text.replace(",", "."))
    variacao = [
        driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[3]/td[2]/span/font').text,
        driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[4]/td[2]/span/font').text,
        driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[5]/td[2]/span/font').text
    ]  

    variacao = [(float(x[:-1].replace(',', '.'))) for x in variacao]

    margem_bruta = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[4]/td[6]/span').text.replace(",", ".").replace("%", ""))
    


    ##### investidor 10 #####
    driver.get('https://investidor10.com.br/acoes/{}/'.format(ação))

    # Define o valor inicial e o passo do loop
    scroll_value = 1000
    scroll_step = 500

    # Define o valor máximo de scroll (ajuste conforme necessário)
    max_scroll_value = 15000

    
    try:
        driver.execute_script('window.scroll(0,3300);')
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="indicators-history"]/button').click()
    except:
        driver.execute_script('window.scroll(0,2300);')
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="indicators-history"]/button').click()


    cagr = [
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[30]/td[3]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[30]/td[4]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[30]/td[5]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[30]/td[6]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[30]/td[7]').text
    ]

    cagr =  list(map(lambda x: int(x.replace("-", "0").replace("%", "").split(',')[0]), cagr))

    dy = [
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[5]/td[3]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[5]/td[4]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[5]/td[5]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[5]/td[6]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[5]/td[7]').text
    ]

    dy = [(float(x[:-1].replace(',', '.'))) for x in dy]

    payout = [
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[6]/td[3]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[6]/td[4]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[6]/td[5]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[6]/td[6]').text,
        driver.find_element(By.XPATH, '//*[@id="table-indicators-history"]/tbody/tr[6]/td[7]').text
    ]

    payout = [(float(x[:-1].replace(',', '.'))) for x in payout]

    # Rolar a página e clicar nos elementos
    if scroll_to_element(driver, scroll_value, max_scroll_value, scroll_step, '//*[@id="info_about"]/div[1]/div/ul/li[2]/span/span[1]/span/span[2]'):
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="info_about"]/div[1]/div/ul/li[2]/select/option[2]').click()

    #governaça 
    mercado = driver.find_element(By.XPATH, '//*[@id="table-indicators-company"]/div[10]/span[2]').text #Segmento de Listagem
    tag_along = float(driver.find_element(By.XPATH, '//*[@id="table-indicators-company"]/div[12]/span[2]').text.replace("-", "0.0").replace(",", ".").replace("%", ""))
    free_float = float(driver.find_element(By.XPATH, '//*[@id="table-indicators-company"]/div[11]/span[2]').text.replace(",", ".").replace("%", ""))
    
    #Tamanho blue chip
    valor_de_mercado = int(driver.find_element(By.XPATH, '//*[@id="table-indicators-company"]/div[1]/span[2]/div[2]').text.replace(".", "").split(' ')[1])
    

    # Rolar a página e clicar nos elementos
    if scroll_to_element(driver, scroll_value, max_scroll_value, scroll_step, '//*[@id="results_table"]/div[1]/div/ul/li[2]/span/span[1]/span/span[2]'):
        driver.find_element(By.XPATH, '//*[@id="results_table"]/div[1]/div/ul/li[2]/select/option[2]').click()

    lucro_liq5 = [
        driver.find_element(By.XPATH, '//*[@id="table-balance-results"]/tbody/tr[5]/td[3]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-results"]/tbody/tr[5]/td[5]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-results"]/tbody/tr[5]/td[8]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-results"]/tbody/tr[5]/td[11]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-results"]/tbody/tr[5]/td[14]/div[2]').text.replace("-", "0.0"),
    ]

    lucro_liq5 = list(map(lambda x: int(x.replace(".", "")), lucro_liq5))

    # Rolar a página e clicar nos elementos
    if scroll_to_element(driver, scroll_value, max_scroll_value, scroll_step, '//*[@id="assets-liabilities"]/div[1]/div/ul/li[2]/span/span[1]/span/span[2]'):
        driver.find_element(By.XPATH, '//*[@id="assets-liabilities"]/div[1]/div/ul/li[2]/select/option[2]').click()

    patri_liq5 = [
        driver.find_element(By.XPATH, '//*[@id="table-balance-sheet"]/tbody/tr[9]/td[2]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-sheet"]/tbody/tr[9]/td[4]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-sheet"]/tbody/tr[9]/td[7]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-sheet"]/tbody/tr[9]/td[10]/div[2]').text.replace("-", "0.0"),
        driver.find_element(By.XPATH, '//*[@id="table-balance-sheet"]/tbody/tr[9]/td[13]/div[2]').text.replace("-", "0.0"),
    ]

    patri_liq5 = list(map(lambda x: int(x.replace(".", "")), patri_liq5))

    roe = [x / y for x, y in zip(lucro_liq5, patri_liq5)]
    
    fundacao = 2024 - int(driver.find_element(By.XPATH, '//*[@id="data_about"]/div[2]/div/div[1]/table/tbody/tr[5]/td[2]').text)

    prejuizo = driver.find_element(By.XPATH, '//*[@id="styled-checkbox-profitable"]').get_attribute('outerHTML')


    driver.get('https://fundamentei.com/br/{}/'.format(ação))
    
    estatal = "não"
    try:
        estatal = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div/div[3]/div[1]/ul/li[1]/div[2]/span[2]').text

    except NoSuchElementException:
        pass # ou continue se quiser pular para a próxima ação

    try:
        ibov = float(driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[4]/div[4]/div/div[2]/h1').text.replace("-", "0.0").replace(",", ".").replace("%", ""))

    except NoSuchElementException:
        ibov = 0 # ou continue se quiser pular para a próxima ação

    # Aplica os critérios para o contador
    print("")
    print(driver.title)
    #1. Rentabilidade e Desempenho:
    contador = aplicar_criterio(all(x > 0.05 for x in roe), contador)
    print(f"O ROE é {roe}")
    print(contador)

    contador = aplicar_criterio(margem_bruta > 40, contador)
    print(f"A margem_bruta é {margem_bruta}")
    print(contador)

    contador = aplicar_criterio("checked" in prejuizo, contador)
    print(f"A empresa nunca teve prejuizo? {"checked" in prejuizo}")
    print(contador)

    contador = aplicar_criterio(ebit > 0, contador)
    print(f"O ebit é {ebit}")
    print(contador)

    #2. Estrutura de Capital:
    contador = aplicar_criterio(div_liq < 0, contador)
    print(f"A div_liq é {div_liq}") 
    print(contador)
    
    contador = aplicar_criterio((abs(lucro_liq) - abs(div_liq)) > 0, contador) # Critério Dívida vs Lucro
    print(f"O lucro_liq vs div_liq é {abs(lucro_liq) - abs(div_liq)}")
    print(contador)

    #3. Crescimento:
    contador = aplicar_criterio(all(x > 5 for x in cagr), contador) 
    print(f"O cagr é {cagr}")
    print(contador)
    
    #4. Valuation:
    contador = aplicar_criterio(0 < pl < 20, contador) #Critério P/L (Preço/Lucro)
    print(f"O pl é {pl}")
    print(contador)
    
    contador = aplicar_2_criterio(0 < p_vp <= 2, 2 < p_vp <= 5, contador) #P/VP (Preço/Valor Patrimonial)
    print(f"O p_vp é {p_vp}")
    print(contador)

    #5. Governança Corporativa:
    contador = aplicar_2_criterio(tag_along >= 80, 50 <= tag_along < 80, contador)
    print(f"O tag_along é {tag_along}")
    print(contador)

    contador = aplicar_2_criterio(free_float >= 80, 40 <= free_float < 80, contador)
    print(f"O free_float é {free_float}")
    print(contador)

    contador = aplicar_criterio(fundacao > 30, contador)  # Critério Fundação
    print(f"A fundação é {fundacao}")
    print(contador)

    contador = aplicar_criterio(estatal != "GOVERNO", contador)  # Critério Estatal
    print(f"O estatal é {estatal}")
    print(contador)

    #6. Dividendos 
    contador = aplicar_2_criterio(all(x > 5 for x in dy), (sum(dy)/len(dy)) > 5, contador) # Critério DY
    print(f"O dy é {dy}")
    print(contador)

    contador = aplicar_2_criterio(all(x > 50 for x in payout), (sum(payout)/len(payout)) > 50, contador) # Critério DY
    print(f"O payout é {payout}")
    print(contador)

    #7. Mercado
    contador = aplicar_2_criterio(all(-15 < x <= 15 for x in variacao), all(-15 < x <= 50 for x in variacao), contador)  # Critério vies
    print(f"A variação é {variacao}")
    print(contador)

    contador = aplicar_criterio(valor_de_mercado > 100000000000 and ibov > 1, contador)
    print(f"O valor_de_mercado é {valor_de_mercado}")
    print(f"O ibov é {ibov}")
    print(contador)
    
    # Salva a nota no DataFrame
    df_resultados = pd.concat([df_resultados, pd.DataFrame({'Ação': [ação], 'Nota': [contador]})], ignore_index=True)

    # Imprime os valores das variáveis
    
    nota_ponderada = calcular_nota_ponderada(roe, cagr)
    nota_simples = calcular_nota_simples(margem_bruta, div_liq, lucro_liq, pl, p_vp, tag_along, free_float, fundacao, estatal, dy, payout, variacao, valor_de_mercado, ibov)
    nota_total = nota_ponderada + nota_simples  # Soma as notas
    
    print(f"Nota ponderada: {nota_ponderada:.2f}")
    print(f"Nota simples: {nota_simples:.2f}")
    print(f"Nota total: {nota_total:.2f}")


# Fecha o navegador
driver.quit()

# Imprime os resultados no final
print("")
print(df_resultados)

df_resultado_ajust = df_resultados.copy()
df_resultado_ajust['Nota'] = df_resultado_ajust['Nota'].apply(lambda x: x if x >= 1 else (0.5 if x < 1 else x))

# Calcula a soma das notas ajustadas
soma = df_resultado_ajust['Nota'].sum()

# Cria o novo DataFrame com as colunas 'Ação' e 'Recomendação'
df_recomendacao = pd.DataFrame({'Ação': df_resultado_ajust['Ação'], 
                                'Recomendação': df_resultado_ajust['Nota'] / soma })

# Imprime os DataFrames
print(df_resultado_ajust)
print(df_recomendacao)

Função para nota considerando pesos para cada critério 